In [1]:
import csv
import re
import math
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("..") # adds higher directory to python modules path

from LoaderPACK.Loader import testload_5min
import torch
import matplotlib.pyplot as plt

from multiprocessing import Process

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class LSTM_net(nn.Module):
    """
    This is a network with the LSTM nodes
    """
    def __init__(self, batch_size, device):
        """
        Args:
            out_channels (int): The amount of channels the output tensor gets.
        """
        super(LSTM_net, self).__init__()


        self.batch_size = batch_size
        self.input_size = 1 # the number of series
        hidden_size = 5 # hyper para

        D = 2 # bc. bi = True
        num_layers = 1 # default (how many cells)


        proj_size = 1 # This allows us to rechive two values
        hout = proj_size # since proj_size > 0

        self.lstm = nn.LSTM(self.input_size, hidden_size, num_layers, \
                            batch_first=True, bidirectional=True, \
                            proj_size = proj_size)
                            # (input_size, hidden)


        self.soft = nn.Softmax(dim=1) # Using sigmoid instead of softmax

    def forward(self, x):
        ind = x.view(self.batch_size, -1, self.input_size)
        out, _ = self.lstm(ind)

        ss = torch.sum(out, 2)
        minusss = 1 - ss

        out = torch.stack((ss, minusss), dim = 1)

        return self.soft(out)

In [3]:
import torch
from typing import Callable, Optional
import torch.nn.functional as F
from torch.nn.modules import Module

class _Loss(Module):
    reduction: str

    def __init__(self, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction: str = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction


class _WeightedLoss(_Loss):
    def __init__(self, weight: Optional[torch.Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_WeightedLoss, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.weight: Optional[Tensor]
        
class CrossEntropyLoss_jav(_WeightedLoss):
    r"""This criterion computes the cross entropy loss between input and target.

    Examples::

        >>> # Example of target with class indices
        >>> loss = nn.CrossEntropyLoss()
        >>> input = torch.randn(3, 5, requires_grad=True)
        >>> target = torch.empty(3, dtype=torch.long).random_(5)
        >>> output = loss(input, target)
        >>> output.backward()
        >>>
        >>> # Example of target with class probabilities
        >>> input = torch.randn(3, 5, requires_grad=True)
        >>> target = torch.randn(3, 5).softmax(dim=1)
        >>> output = loss(input, target)
        >>> output.backward()
    """
    __constants__ = ['ignore_index', 'reduction', 'label_smoothing']
    ignore_index: int
    label_smoothing: float

    def __init__(self, weight: Optional[torch.Tensor] = None, size_average=None, ignore_index: int = -100,
                 reduce=None, reduction: str = 'mean', label_smoothing: float = 0.0) -> None:
        super(CrossEntropyLoss_jav, self).__init__(weight, size_average, reduce, reduction)
        self.ignore_index = ignore_index
        self.label_smoothing = label_smoothing

    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        return F.cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction,
                               label_smoothing=self.label_smoothing)

In [4]:
batch_size = 1

device = "cpu"

if str(device) == "cpu":
    fl = torch.FloatTensor
    it = torch.LongTensor
else:
    fl = torch.cuda.FloatTensor
    it = torch.cuda.LongTensor

model = LSTM_net(batch_size=batch_size, device=device).to(device)

ind = torch.randn(1, 1, 60000)
tar = torch.randint(0, 1, (1, 1, 60000))

In [5]:
lossFunc = CrossEntropyLoss_jav(weight = torch.tensor([1., 5.]).to(device), reduction = "mean")

In [6]:
model.zero_grad()
y_pred = model(ind)
target = tar.view(-1).type(it)
pred = y_pred.transpose(1, 2).reshape(-1, 2).type(fl)

In [7]:
import time

In [8]:
st = time.time()
loss = lossFunc(pred, target)
loss.backward()
en = time.time()
print(loss.item())
print(en-st)

0.9765810370445251
79.92066240310669


In [9]:
lossFunc = nn.CrossEntropyLoss(weight = torch.tensor([1., 5.]).to(device), reduction = "mean")

In [10]:
model.zero_grad()
y_pred = model(ind)
target = tar.view(-1).type(it)
pred = y_pred.transpose(1, 2).reshape(-1, 2).type(fl)

In [11]:
st = time.time()
loss = lossFunc(pred, target)
loss.backward()
en = time.time()
print(loss.item())
print(en-st)

0.9765810370445251
78.8082058429718


In [193]:
#PyTorch
BETA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
CE_RATIO = 0.5 # weighted contribution of modified CE loss compared to Dice loss

class ComboLoss(nn.Module):
    def __init__(self):
        super(ComboLoss, self).__init__()

    def forward(self, inputs, targets, cd_ratio = CE_RATIO, smooth=1, beta=BETA, eps=1e-9):
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        intersection = (inputs * targets).sum()    
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        
        inputs = torch.clamp(inputs, eps, 1.0 - eps)       
        out = - (beta * ((targets * torch.log(inputs)) + ((1 - beta) * (1.0 - targets) * torch.log(1.0 - inputs))))
        weighted_ce = out.mean(-1)
        combo = (cd_ratio * weighted_ce) - ((1 - cd_ratio) * dice)
        print("dice", dice)
        return combo

In [208]:
lossFunc = ComboLoss()

lol = torch.Tensor([0, 1, 0])
jo = torch.Tensor([0.0, 1, 0.])
print(lol)
print(jo)
print(lossFunc(jo, lol))
val = lossFunc(jo, lol)

tensor([0., 1., 0.])
tensor([0., 1., 0.])
dice tensor(1.)
tensor(nan)
dice tensor(1.)


In [211]:
torch.tensor([1, float('nan'), 2])

tensor(True)

In [205]:
print(val)

tensor(nan)


In [207]:
if val:
    print("True")

True


In [22]:
l = torch.Tensor([[0, 0], [1, 1]])
l.mean()

tensor(0.5000)

In [23]:
l.mean(-1)

tensor([0., 1.])

In [24]:
model.zero_grad()
y_pred = model(ind)
target = tar.view(-1).type(it)
pred = y_pred[0][1]

In [57]:
lossFunc = ComboLoss()

In [26]:
st = time.time()
loss = lossFunc(pred, target)
loss.backward()
en = time.time()
print(loss.item())
print(en-st)

0.17442743480205536
82.47842001914978


In [164]:
lol = torch.randint(0, 2, (1, 10))
jo = torch.randn(10).abs()/10
print(lol)
print(jo)
print(lossFunc(jo, lol))

tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])
tensor([0.0910, 0.0255, 0.1250, 0.1495, 0.0041, 0.1771, 0.0728, 0.0398, 0.0161,
        0.0548])
tensor(-0.1948)


tensor([0., 1., 0.])
tensor([0.0000, 0.9999, 0.0000])
tensor(-0.5000)


tensor(nan)

In [51]:
jav = torch.randn(5, 2, 5)


In [52]:
jav

tensor([[[-0.8002, -0.0632, -0.4482, -0.0820,  1.3261],
         [-0.3362, -0.2082,  0.5069, -0.0451,  0.2283]],

        [[ 0.7799,  1.0527, -1.0746, -1.6134,  0.4359],
         [ 0.8401,  2.0553, -0.7265,  1.0813, -0.3151]],

        [[ 0.1770,  0.1140,  0.2830, -0.0796,  0.7359],
         [-0.2245, -1.1063,  1.3846, -0.5966,  1.1795]],

        [[ 0.0573, -0.2319,  0.9246, -1.0776,  1.8473],
         [ 2.0416, -0.2872,  0.4917,  0.9460,  1.3541]],

        [[ 1.4323, -1.1046,  1.1288,  0.1689,  0.7143],
         [-1.6627, -0.0575,  0.8594,  0.4277,  1.2272]]])

In [55]:
jav[:, 1].reshape(-1)

tensor([-0.3362, -0.2082,  0.5069, -0.0451,  0.2283,  0.8401,  2.0553, -0.7265,
         1.0813, -0.3151, -0.2245, -1.1063,  1.3846, -0.5966,  1.1795,  2.0416,
        -0.2872,  0.4917,  0.9460,  1.3541, -1.6627, -0.0575,  0.8594,  0.4277,
         1.2272])